In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

# Data files full clean

In [12]:
import os
import re

if not os.path.exists("./cleandata"):
    os.makedirs("./cleandata")
content = os.listdir("./newdata")

for item in content:
    # delete the text files and the urls files to clean the library
    print(item)
    if item.find(".txt") > 0:
        # print(item)
        with open("./newdata/{}".format(item), "r", encoding="iso-8859-1") as f:
            lines = f.readlines()
        # if (os.path.exists("./cleandata/{}".format(item))):
        with open("./cleandata/{}".format(item), "w+") as f:

            for line in lines:
                if "10 Messungen" in line.strip("\n"):
                    line = line[line.find("Ch") :]
                    # print(newline)
                # if (line.strip("\n") != "þStartfischer1 p33-6 hf8 fc6x 0.9km")
                if "Ch" in line.strip("\n") or "V" in line.strip("\n"):
                    f.write(line.replace(";", ""))
f.close()

# print(content)

fischer2_p33_6_hf8_fc6x_4km.txt
madshus_base7_ts_blue_ clockwise 45 turn.txt
fischer1_p33_6_hf8_fc6x_4km.txt
madshus_base7_ts_blue_point.txt
madshus_base7_ts_blue.txt
fischer_p33_6_hf8_fc6x.txt
fischer2_p33_6_hf8_fc6x.txt
fischer1_p33_6_hf8_fc6x_0_9km.txt
fischer2_p33_6_hf8_fc6x_0_9km.txt


In [20]:
a = {}

for x in content:
    print(x)
    df = pd.read_csv(
        "./newdata/{}".format(x),
        names=["channal", "min", "max", "x", "y"],
        sep="\s+",
        encoding="iso-8859-1",
    )
    dx = df.drop(["x", "y"], axis=1)
    print(df.head())
    # dx['min']= dx['min'].map(lambda x: x.lstrip('+-;').rstrip('aAbBcC;'))
    # dx['channal']= dx['channal'].map(lambda x: x.lstrip(';').rstrip(';'))
    # #idx=dx.index();
    # if 100 in dx.index:
    #     dx=dx.drop(100);
    # dx['min']=pd.to_numeric(dx['min'])
    # dx['max']=pd.to_numeric(dx['max'])
    # dx.index.name=x
    # a[x]=dx
    # print(a[x].head())
    # break;

fischer2_p33_6_hf8_fc6x_4km.txt
           channal      min   max    x    y
fischer2     p33-6      hf8  fcx6  4.0   km
Ch0:;     4433955;  6042328   NaN  NaN  NaN
Ch1:;     4432469;  5269910   NaN  NaN  NaN
Ch2:;     4404755;  5332808   NaN  NaN  NaN
Ch3:;     4400764;  4990275   NaN  NaN  NaN
madshus_base7_ts_blue_ clockwise 45 turn.txt
                  channal   min  max    x           y
madshus base7          TS  blue   no  wax  directions
Ch0:;   4436328;  5838693   NaN  NaN  NaN         NaN
Ch1:;   4438486;  5211841   NaN  NaN  NaN         NaN
Ch2:;   4406715;  5393239   NaN  NaN  NaN         NaN
Ch3:;   4405022;  5047324   NaN  NaN  NaN         NaN
fischer1_p33_6_hf8_fc6x_4km.txt
                 channal      min   max    x    y
þStartfischer1     p33-6      hf8  fc6x  4.0   km
Ch0:;           4377246;  6136439   NaN  NaN  NaN
Ch1:;           4361919;  5299389   NaN  NaN  NaN
Ch2:;           4347099;  5412315   NaN  NaN  NaN
Ch3:;           4336738;  5034490   NaN  NaN  NaN
mad

In [ ]:
# df=pd.read_csv("./newdata/fischer1_p33_6_hf8_fc6x_4km.txt",names=['channal','min','max','x','y'],sep='\s+',engine='python',encoding='iso-8859-1',skiprows=1)
# dx=df.drop(['x', 'y'], axis=1)
# dx['min']= dx['min'].map(lambda x: x.lstrip('+-;').rstrip('aAbBcC;'))
# dx['channal']= dx['channal'].map(lambda x: x.lstrip(';').rstrip(';'))
# #dx=dx.drop(100);
# dx['min']=pd.to_numeric(dx['min'])
# dx['max']=pd.to_numeric(dx['max'])
# dx.index.name="fischer1_p33_6_hf8_fc6x_4km"
# dx.to_excel("formatet.xlsx");
# dx.head(10)

In [5]:
print("hallo Wolrd!")

hallo Wolrd!


NameError: name 'content' is not defined